<a href="https://colab.research.google.com/github/bombermal/Personal_Proj_Consume_Analyzer/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Groceries analyzer
## Description

This algorithm aim to analyze groceries purchase, price and his variations.

The project is divided in 5 parts:
1. Aquire data
  - [ ] PDF ( nota potiguar, Manual - Could web scrapping be a better option?)
2. Load data in
  - [X] Load PDF
    - [X] Read table
    - [X] Read store information
    - [X] Read data/time information
3. Store data
  - [ ] Store data in MySql
    - [X] Install docker
     - [X] Create container    
    - [ ] Install MySqlWorkbench
      - [ ] Create table
    - [ ] Ipybn
      - [ ] Connect to main.ipynb

4. Process data
  - [ ] Group stores
  - [ ] Group products by category ( categoryze by processment type )
5. Show results
  - [ ] Show price variation ( Variation, man, seasonality )
  - [ ] Show costly categories
  - [ ] Rank products
6. Posterior ideas
  - [ ] Suggest substitutes based on price seasonality
  - [ ] ML for actegorization?

# Imported libraries

In [1]:
# Install libraries
!pip install tabula-py PyMuPDF
# !pip install tika 

     |████████████████████████████████| 12.0 MB 2.3 MB/s 


In [252]:
# Import
import os
import tabula
import fitz  # this is pymupdf
import pandas as pd

Set work directory

In [10]:
cd /content/drive/MyDrive/Colab\ Notebooks/2022/Personal_Proj_Consume_Analyzer

/content/drive/MyDrive/Colab Notebooks/2022/Personal_Proj_Consume_Analyzer


Manually - Confirm directory

In [11]:
ls

 Data/   main   README.md  'Scrap Nota Potiguar.ipynb'


# Functions

In [12]:
def listFilesByYears(folder, years = 2020):
  """
    Load a single or several files at same time

    @Params
    folder: string -> Complete file path
    years: str/list -> String to choose a single file or a list to load several

    @Return
    dictionary -> 'Folder Name'( Key ) : List of files names in folder( value )
  """
  respDict = {}
  if isinstance(years, list):
    respDict = {yr: os.listdir(folder+yr) for yr in years}
  else:
    try:
      respDict = {years: os.listdir(folder+str(years))}
    except FileNotFoundError:
      print('Não existe dados para o ano escolhido')

  return respDict

def loadPdf(completeFilePath):
  # Load file
  file = fitz.open(completeFilePath)
  page = file.load_page(0)

  # Select Store and Buyer information based in the coordinates of 'Emitente' and 'Vl. total'
  x1, y1 = page.search_for("Emitente")[0][:2]
  xF, y2 = page.search_for("Vl. total")[0][2:]

  # y1 = top
  # x1 = left
  # y2 = top + height
  # x2 = left + width

  headTable = tabula.read_pdf(completeFilePath, pages = 1, pandas_options = {'header': None}, area = [y1, x1, y2, xF ])
  # Split row with a '|' in two
  headTable = headTable[0][0].str.split('|').explode([0]).reset_index(drop = True).to_frame()
  # Split column in two, using ':' as separator
  headTable = headTable[0].str.split(':', 1, expand = True)
  # Rename columns
  headTable = headTable.rename(columns={0: 'Variables', 1: 'Values'})
  # Select non-empty rows
  mask = headTable['Values'].astype(bool)
  # Ready to use DataFrame
  headTable = headTable[mask]

  # Select Date, Time and other buy information based in the coordinates of 'Valor Total', 'Autorizada' and 'Vl. total' from previous selection
  x1, y1 = page.search_for("Valor Total")[0][:2]
  y2 = page.search_for("Autorizada")[0][1]

  footTable = tabula.read_pdf(completeFilePath, pages = 1, pandas_options = {'header': None}, area = [y1, x1, y2, xF ])
  # Fix DataFrame
  bottonHalf = footTable[0][1].str.split(':', 1, expand = True).dropna(how = 'all').dropna(how = 'any')
  topHalf = footTable[0].drop(labels = [1], axis = 1).dropna(how = 'all').rename(columns = {2 : 1})
  footTable = pd.concat([topHalf, bottonHalf])
  # Rename columns
  footTable = footTable.rename(columns={0: 'Variables', 1: 'Values'})

  # Select Groceries items buy information based in the coordinates of 'Item', 'Valor Total' and 'Vl. total' from previous selection
  x1, y1 = page.search_for("Item")[0][:2]
  y2 = page.search_for("Valor Total")[0][1]
  
  groceriesTable = tabula.read_pdf(completeFilePath, pages = 1, area = [y1, x1, y2, xF ])
  groceriesTable = groceriesTable[0].drop('Item', axis=1).applymap(lambda x: x.replace(',', '.'))
  groceriesColumns = groceriesTable.columns.tolist()
  groceriesIntColumns = [groceriesColumns[1]] + groceriesColumns[3:]

  groceriesTable[groceriesIntColumns] = groceriesTable[groceriesIntColumns].apply(lambda x: pd.to_numeric(x, downcast='float'))

  return headTable, groceriesTable, footTable

# List PDF files

Here we can search the directories and create a dictionary with all file's names in each directory

In [36]:
# Root folder
rootFolderPath = '/content/drive/MyDrive/Colab Notebooks/2022/Personal_Proj_Consume_Analyzer/Data/'
# List of years in Data
yearsFoldersList = os.listdir(rootFolderPath)

listFilesByYears(rootFolderPath, yearsFoldersList)

{'2020': ['1.pdf',
  '7.pdf',
  '9.pdf',
  '5.pdf',
  '6.pdf',
  '8.pdf',
  '4.pdf',
  '3.pdf',
  '2.pdf']}

# Load files in memory

Load pdf

In [425]:
# import single file
head, mid, foot = loadPdf(rootFolderPath+'2020/1.pdf')
display(head)
display(mid)
display(foot)

,Variables,Values
1,RAZÃO SOCIAL,CARREFOUR COMERCIO E INDUSTRIA LTDA
2,CNPJ,45.543.915/0068-99
3,IE,20.078.271-1
4,ENDEREÇO,"RDV BR 101, SN, LAGOA NOVA, NATAL, RN, 59063-904"
7,CPF,082.707.764-59


,Descrição,Qtde.,Unid.,Vl. unid.,Vl. total
0,BLUE SPIRIT ICE 275M,6.000,un,3.990000,23.940001
1,CARNE MOIDA COXAO MO,0.714,kg,34.900002,24.920000
2,QJ MUSS FAT SADIA kg,0.302,kg,43.790001,13.220000
3,BACON DEF SADIA,0.378,kg,23.990000,9.070000
4,QUEIJO PARMESAO IMPO,0.180,kg,89.989998,16.200001
5,MOLHO TOM POMAROLA T,1.000,un,2.290000,2.290000
6,EXT TO ELEF ERV 130G,1.000,un,1.890000,1.890000
7,BANANA PRATA COMUM K,1.540,kg,2.490000,3.830000


,Variables,Values
0,Valor Total dos Produtos (R$),"95,36"
1,Valor Descontos (R$),"0,00"
2,Valor Pago (R$),"95,36"
3,Forma Pagamento,Cartão de Crédito
6,Data de Emissão,31/01/2020 12:59:10
7,Data de Autorização,31/01/2020 12:59:13
8,Protocolo,324200028247148


In [ ]:
from tika import parser # pip install tika

raw = parser.from_file(file1)
print(raw['content'])